# 머신러닝 실습(회귀)
- section 1 : 문제정의
- section 2 : 라이브러리 및 데이터 불러오기
- section 3 : 탐색적데이터분석(EDA)
- section 4 : 데이터 전처리
- section 5 : 검증 데이터 나누기
- section 6 : 머신러닝 학습 및 평가
- section 7 : 예측 및 결과 파일 생성

## 1. 문제정의
- 10개의 아울렛 매장에서 1,500여개 제품에 대한 판매 데이터를 수집했다.
- 예측 모델을 만들고 아울렛 특정 매장에서 각 제품의 판매금액을 예측하시오.

    - 평가 기준은 RMSE로 평가
    - label(target)은 판매금액(Item_Outlet_Sale)
    - 제출 파일은 예측값만 result.csv 파일로 생성해 제출(컬럼명 : pred , 1개)

## 2. 라이브러리 및 데이터 불러오기

In [28]:
import pandas as pd
train = pd.read_csv('./ch5_data/train.csv')
test = pd.read_csv('./ch5_data/test.csv')


## 3. 탐색적 데이터 분석(EDA)
- 데이터의 크기를 확인한다.

- train 데이터 (6818, 12)
- test 데이터(1705, 11)

In [29]:
print(train.shape)
print(test.shape)

(10000, 21)
(10000, 20)


### 데이터 샘플 확인
- 카테고리(문자)와 숫자 컬럼이 혼합되어있는 것을 확인
- 마지막 컬럼 Item_outlet_Sales는 target(label) 컬럼인데, 금액이고 소수점있는 숫자.

In [30]:
train.head()

,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly,Monthly_Balance,Credit_Score,Credit_Mix,...,Age,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Monthly_Inhand_Salary,Changed_Credit_Limit,Outstanding_Debt,Total_EMI_per_month
0,56.0,16.0,11.0,35.598217,120.0,Yes,229.093478,252.385965,1,Bad,...,15.0,36597.56,8.0,10.0,29.0,5.0,3143.796667,22.49,2963.18,122.900223
1,49.0,23.0,12.0,25.553106,120.0,Yes,104.613906,219.105944,1,Bad,...,28.0,32057.30,9.0,8.0,16.0,7.0,2606.441667,1.40,1327.26,164.859426
2,34.0,20.0,6.0,40.039954,174.0,Yes,338.626965,251.265589,1,Bad,...,46.0,75868.80,6.0,10.0,32.0,7.0,6074.400000,3.60,1432.71,297.547446
3,21.0,13.0,8.0,25.711678,143.0,NM,116.816864,259.927960,2,Standard,...,46.0,17092.69,7.0,3.0,19.0,7.0,1695.390833,16.40,1417.06,62.794260
4,19.0,13.0,6.0,39.140463,138.0,Yes,87.262887,626.212330,1,Bad,...,45.0,81471.96,6.0,6.0,25.0,5.0,6763.330000,27.09,2679.69,202.857783


### 데이터 자료형 확인
- float형 4개
- int형 1개
- object 형 7개

In [31]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Delay_from_due_date       10000 non-null  float64
 1   Num_of_Delayed_Payment    10000 non-null  float64
 2   Num_Credit_Inquiries      10000 non-null  float64
 3   Credit_Utilization_Ratio  10000 non-null  float64
 4   Credit_History_Age        10000 non-null  float64
 5   Payment_of_Min_Amount     10000 non-null  object 
 6   Amount_invested_monthly   10000 non-null  float64
 7   Monthly_Balance           10000 non-null  float64
 8   Credit_Score              10000 non-null  int64  
 9   Credit_Mix                10000 non-null  object 
 10  Payment_Behaviour         10000 non-null  object 
 11  Age                       10000 non-null  float64
 12  Annual_Income             10000 non-null  float64
 13  Num_Bank_Accounts         10000 non-null  float64
 14  Num_Cre

### 기술 통계 확인
- describe()
---
- median(50%)>mean인 경우 왼쪽 꼬리가 긴 분포
- median(50%)<mean인 경우 오른쪽 꼬리가 긴 분포
---
- Item_Visibility, Item_Outlet_Sales 컬럼은 Median(50%)<mean 인 데이터모양이 오른쪽꼬리가 긴 분포모양

In [32]:
train.describe()

,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Utilization_Ratio,Credit_History_Age,Amount_invested_monthly,Monthly_Balance,Credit_Score,Age,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Monthly_Inhand_Salary,Changed_Credit_Limit,Outstanding_Debt,Total_EMI_per_month
count,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,21.263100,13.381300,5.79400,32.306216,220.501400,195.497580,400.130732,1.880700,33.33680,50318.451889,5.382900,5.538000,14.468200,3.525700,4182.906254,10.435633,1428.996096,105.114322
std,14.778976,6.217876,3.86772,5.093599,99.204263,196.150962,211.034108,0.679789,10.73122,38171.440801,2.566271,2.096186,8.687977,2.441585,3175.891720,6.501954,1158.852422,125.502925
min,0.000000,0.000000,0.00000,20.719745,2.000000,0.000000,0.007760,1.000000,14.00000,7006.035000,0.000000,0.000000,1.000000,0.000000,303.645417,0.500000,0.230000,0.000000
25%,10.000000,9.000000,3.00000,28.075839,144.000000,74.056997,268.741936,1.000000,24.00000,19259.307500,4.000000,4.000000,7.000000,2.000000,1626.342083,5.507500,562.340000,28.588769
50%,18.000000,14.000000,6.00000,32.310153,219.000000,130.089440,334.854343,2.000000,33.00000,36875.140000,5.000000,5.000000,13.000000,3.000000,3073.272917,9.410000,1168.235000,65.971478
75%,28.000000,18.000000,8.00000,36.491502,301.000000,234.960725,470.351899,2.000000,42.00000,71507.587500,7.000000,7.000000,20.000000,5.000000,5936.870000,14.630000,1960.420000,144.429302
max,67.000000,28.000000,17.00000,47.103409,404.000000,1674.337665,1567.208309,3.000000,56.00000,179987.280000,10.000000,11.000000,34.000000,9.000000,15167.180000,29.980000,4998.070000,1701.955013


### Object 컬럼의 unique 개수 파악
- 3개부터 많게는 1554개까지 분포
- Item_Identifier은 1,554개
---
- unique의 개수에 따라 원-핫 인코딩을 사용하거나 레이블 인코딩을 사용하기도 한다
- 1,5554개를 원-핫 인코딩을 했을 때는 컬럼수가 엄청나게 늘어나고 대부분의 값이 0으로 대체되며 낭비가 심하다.

In [33]:
train.describe(include='O')

,Payment_of_Min_Amount,Credit_Mix,Payment_Behaviour
count,10000,10000,10000
unique,3,3,6
top,Yes,Standard,Low_spent_Small_value_payments
freq,5269,4591,3416


### test 데이터셋 분포 확인
- Item_Identifier를 제외하고 모두 train 데이터 셋과 unique 개수가 동일

In [34]:
test.describe(include='O')

,Payment_of_Min_Amount,Credit_Mix,Payment_Behaviour
count,10000,10000,10000
unique,3,3,6
top,Yes,Standard,Low_spent_Small_value_payments
freq,5167,4590,3498


### 결측치 확인
- train Data
    - Item_Weight 1162개 
    - Outlet_Size는 1940개
---
- test Data
    - Item_Weight 301개 
    - Outlet_Size는 470개

In [35]:
train.isnull().sum()

Delay_from_due_date         0
Num_of_Delayed_Payment      0
Num_Credit_Inquiries        0
Credit_Utilization_Ratio    0
Credit_History_Age          0
Payment_of_Min_Amount       0
Amount_invested_monthly     0
Monthly_Balance             0
Credit_Score                0
Credit_Mix                  0
Payment_Behaviour           0
Age                         0
Annual_Income               0
Num_Bank_Accounts           0
Num_Credit_Card             0
Interest_Rate               0
Num_of_Loan                 0
Monthly_Inhand_Salary       0
Changed_Credit_Limit        0
Outstanding_Debt            0
Total_EMI_per_month         0
dtype: int64

In [36]:
test.isnull().sum()

Delay_from_due_date         0
Num_of_Delayed_Payment      0
Num_Credit_Inquiries        0
Credit_Utilization_Ratio    0
Credit_History_Age          0
Payment_of_Min_Amount       0
Amount_invested_monthly     0
Monthly_Balance             0
Credit_Mix                  0
Payment_Behaviour           0
Age                         0
Annual_Income               0
Num_Bank_Accounts           0
Num_Credit_Card             0
Interest_Rate               0
Num_of_Loan                 0
Monthly_Inhand_Salary       0
Changed_Credit_Limit        0
Outstanding_Debt            0
Total_EMI_per_month         0
dtype: int64

## 4. 데이터 전처리


In [37]:
target = train.pop("Credit_Score")
cols = train.columns[train.dtypes=='object']
cols

Index(['Payment_of_Min_Amount', 'Credit_Mix', 'Payment_Behaviour'], dtype='object')

In [38]:
train['Payment_of_Min_Amount'] = train['Payment_of_Min_Amount'].astype('category')
train['Credit_Mix'] = train['Credit_Mix'].astype('category')
train['Payment_Behaviour'] = train['Payment_Behaviour'].astype('category')

test['Payment_of_Min_Amount'] = test['Payment_of_Min_Amount'].astype('category')
test['Credit_Mix'] = test['Credit_Mix'].astype('category')
test['Payment_Behaviour'] = test['Payment_Behaviour'].astype('category')

## 5. 검증 데이터 나누기


In [39]:
!pip install scikit-learn

In [40]:
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val = train_test_split(
    train,target,test_size=0.2, random_state=0
)

X_train.shape,X_val.shape,y_train.shape,y_val.shape


((8000, 20), (2000, 20), (8000,), (2000,))

In [41]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train,y_train)

ValueError: could not convert string to float: 'Yes'

In [ ]:
# !pip install lightgbm

In [42]:
import lightgbm as lgb
lgbmc = lgb.LGBMClassifier(random_state=0, verbose=1)
lgbmc.fit(X_train,y_train)
pred = lgbmc.predict(X_val)

pred

ModuleNotFoundError: No module named 'Lightgbm'